# Fall Detection for Elderly Wearable Tech

### Data Collection and Preparation
####  Data Sources
The dataset used in this project consists of time-series data collected from wearable devices, specifically accelerometer and gyroscope sensors. The sensors operate at a frequency of 50Hz, capturing detailed motion data. The data contains multiple fall scenarios along with routine movements, which helps in training the model to recognize fall patterns.
#### Data Merging and Cleaning
To combine the accelerometer and gyroscope data, the datasets were merged on the timestamp for each activity. Gyroscope time lists were discarded to avoid redundancy, and missing values were carefully handled by removing NaNs, ensuring data integrity for the model.

In [15]:
import os
import pandas as pd
import numpy as np


# Function to merge accel and gyro data
def merge_data(accel_file, gyro_file):
    accel_data = pd.read_csv(accel_file)
    gyro_data = pd.read_csv(gyro_file)
    merged_data = pd.concat([accel_data, gyro_data.drop(columns=['gyro_time_list'])], axis=1)  # drop gyro_time_list
    return merged_data

# Function to label the merged data based on timestamps
def apply_labels(merged_data, filename, fall_timestamps):
    # Filter rows for the current filename
    file_timestamps = fall_timestamps[fall_timestamps['filename'] == filename]

    # Initialize the labels column with 0
    merged_data['labels'] = 0

    # Loop through each timestamp entry for this file
    for _, row in file_timestamps.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']

        # Assign label 1 if accel_time_list falls within the start_time and end_time
        merged_data.loc[(merged_data['accel_time_list'] >= start_time) & (merged_data['accel_time_list'] <= end_time), 'labels'] = 1

    return merged_data

# Load the fall timestamps data
"fall_timestamps_path = 'path_to_dataset/fall_timestamps.csv'\n"
fall_timestamps = pd.read_csv(fall_timestamps_path)

# Base directory containing F01, F02, ... folders
"base_dir = 'path_to_dataset/50hz/'\n",

# Prepare to store all merged data with user and fall information
all_data = []

# Iterate over F01, F02, ..., F08 folders only
for folder in os.listdir(base_dir):
    if folder.startswith('F'):  # Ensure we are working only on F folders (fall types)
        folder_path = os.path.join(base_dir, folder)

        # Extract the fall type from the folder name (e.g., F01)
        fall_type = folder

        # Iterate over the files in each folder
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if 'accel.csv' in file:
                    # Identify corresponding gyro file
                    gyro_file = file.replace('accel.csv', 'gyro.csv')
                    
                    accel_file_path = os.path.join(root, file)
                    gyro_file_path = os.path.join(root, gyro_file)
                    
                    if os.path.exists(gyro_file_path):
                        # Merge accel and gyro data
                        merged_data = merge_data(accel_file_path, gyro_file_path)
                        
                        # Get the filename format used in the fall_timestamps file (e.g., F01/U01_R01)
                        filename = f"{folder}/{file.replace('_accel.csv', '')}"
                        
                        # Extract user ID (e.g., U01) from the file name
                        user_id = file.split('_')[0]

                        # Apply labels to the merged data based on the timestamps
                        merged_data = apply_labels(merged_data, filename, fall_timestamps)
                        
                        # Add user and fall type columns to the merged data
                        merged_data['user_id'] = user_id
                        merged_data['fall_type'] = fall_type
                        
                        # Append the merged data to the all_data list
                        all_data.append(merged_data)

                        # Optionally, remove the original files after merging
                        os.remove(accel_file_path)
                        os.remove(gyro_file_path)

print("All files merged and labeled successfully.")

# Step 2: Combine all merged data into one final dataset
final_merged_data = pd.concat(all_data, axis=0)

# Save the final merged dataset to a CSV file
final_output_path = 'path_to_dataset/final_merged_dataset_with_users_fall.csv'
final_merged_data.to_csv(final_output_path, index=False)
print("Final merged dataset saved with user and fall type information.")

All files merged and labeled successfully.
Final merged dataset saved with user and fall type information.
